# Google Vision API サンプル

tesseractを用いてOCRを行っているが，コンテナサイズの最適化のためにVisionAPIが使えるか試してみる
クレカ紐づけが必要なので，再度，紐づけ完了したら実験する

## 使い方
1. config.pyを作成
2. config.pyの中に"API_KEY"という変数を用意し，その中にAPIKEYを記述（githubの管理外にしている）
3. 後はnotebookの実行を進める

In [ ]:
# API_KEYを記述
import config

In [ ]:
from base64 import b64encode
import json
import requests
import glob
import re

In [ ]:
ENDPOINT = 'https://vision.googleapis.com/v1/images:annotate'

In [ ]:
def numerical_sort(value):
    numbers = re.compile(r"(\d+)")
    parts = numbers.split(value)
    parts[1::2] = map(int, parts[1::2])
    return parts

def glob_sorted_with_number(path):
    return sorted(glob.glob(path), key=numerical_sort)

In [ ]:
images = glob_sorted_with_number('output-*.png')
images

In [ ]:
img_requests = []
for img in images:
    with open(img, 'rb') as f:
        content = b64encode(f.read()).decode()
        form = {'image': {'content': content},
                'features': [{'type': 'TEXT_DETECTION',
                              'maxResults': 5
                             }]
               }
        img_requests.append(form)

In [ ]:
response = requests.post(ENDPOINT,
                         data=json.dumps({'requests': img_requests}).encode(),
                         params={'key':config.API_KEY},
                         headers={'Content-Type': 'application/json'})

In [ ]:
response.status_code

In [ ]:
for idx, resp in enumerate(response.json()['responses']):
    print(json.dumps(resp, indent=2))